In [ ]:
import requests as req
import pandas as pd

import logging
import sys
import os
import json
logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

# Authenticate to RW API

In [ ]:
from configparser import ConfigParser
config = ConfigParser()
config.read("/Users/nathansuberi/Desktop/WRI_Programming/cred/.env")
api_token = config.get("auth", "rw_api_token")

AUTH_TOKEN = api_token # <Insert Auth Token Here>

# Viewing current data set 
Goal is to switch all widgets from this old data set to a new one with a different table name

In [ ]:
rw_id = 'c61c364b-1d68-4dd9-ae3d-76c2a0022280'

# Base URL for getting dataset metadata from RW API
url = "https://api.resourcewatch.org/v1/dataset/{rw_id}?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"
url = url.format(rw_id = rw_id)
# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
try:
    data = res.json()["data"]
except:
    raise Exception('No data available')
    
### Convert the json object returned by the API into a pandas DataFrame
atts = data["attributes"]
metadata = atts["metadata"]
layers = atts["layer"]
widgets = atts["widget"]
tags = atts["vocabulary"]

api_info = {
    "name":atts["name"],
    "table_name":atts["tableName"],
    "provider":atts["provider"],
    "date_updated":atts["updatedAt"],
    "num_metadata":len(metadata),
    "metadata": metadata,
    "num_layers":len(layers),
    "layers": layers,
    "num_widgets":len(widgets),
    "widgets": widgets,
    "num_tags":len(tags),
    "tags":tags
}

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
old_ds_info = pd.DataFrame.from_dict(api_info, orient='index')
old_ds_info.columns = [rw_id]
old_table_name = old_ds_info.loc['table_name'].values[0]
old_ds_info

# Create a dataset using API

In [ ]:
headers = {
    'content-type': "application/json",
    'authorization': "Bearer {}".format( AUTH_TOKEN )
}

ds_specs = {
    "connectorType":"rest",
    "provider":"cartodb",
    "connectorUrl":"https://wri-rw.carto.com/tables/com_009_flowmfa",
    "application":["rw"],
    "name":"Material Extraction, Trade, and Use"
}

create_res = req.request("POST", 
                  'https://api.resourcewatch.org/v1/dataset', 
                  data=json.dumps(ds_specs), 
                  headers = headers)

In [ ]:
ds_res = create_res.json()['data']

new_rw_id = ds_res['id']
new_table_name = ds_res['attributes']['tableName']

ds_res

In [ ]:
url = "https://api.resourcewatch.org/v1/dataset/{rw_id}/{endpoint}"  

for metadata in old_ds_info.loc['metadata'].values[0]:
    metadata['attributes']['language'] = 'en'
    metadata['attributes']['application'] = 'rw'
    print(metadata['attributes'])
    _url = url.format(rw_id = new_rw_id, endpoint = 'metadata')
    res = req.request("POST", _url, data=json.dumps(metadata['attributes']), headers = headers)
    logging.info(res.text)    
        
for widget in old_ds_info.loc['widgets'].values[0]:
    widget['attributes']['application'] = ['rw']
    widget['attributes']['language'] = 'en'
    _url = url.format(rw_id = new_rw_id, endpoint = 'widget')
    res = req.request("POST", _url, data=json.dumps(widget['attributes']), headers = headers)
    logging.info(res.text)   
    


In [ ]:
logging.info('OLD ID: {}'.format(rw_id))
logging.info('NEW ID: {}'.format(new_rw_id))


# Base URL for getting dataset metadata from RW API
url = "https://api.resourcewatch.org/v1/dataset/{rw_id}?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"
url = url.format(rw_id = new_rw_id)
# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
try:
    data = res.json()["data"]
except:
    raise Exception('No data available')
    
### Convert the json object returned by the API into a pandas DataFrame
atts = data["attributes"]
metadata = atts["metadata"]
layers = atts["layer"]
widgets = atts["widget"]
tags = atts["vocabulary"]

api_info = {
    "name":atts["name"],
    "table_name":atts["tableName"],
    "provider":atts["provider"],
    "date_updated":atts["updatedAt"],
    "num_metadata":len(metadata),
    "metadata": metadata,
    "num_layers":len(layers),
    "layers": layers,
    "num_widgets":len(widgets),
    "widgets": widgets,
    "num_tags":len(tags),
    "tags":tags
}

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
new_ds_info = pd.DataFrame.from_dict(api_info, orient='index')
new_ds_info.columns = [new_rw_id]
new_ds_info

# Create new layers

In [ ]:
new_ds_info.loc['layers'].values[0]

In [ ]:
# Read in data
data = cc.read('com_009_flowmfa')

# Read in flows
flows = cc.read('com_009_flows')

# Read in mfa13
mfa13 = cc.read('com_009_mfa13')

# Read in mfa4
mfa4 = cc.read('com_009_mfa4')

# Loop over years
years = data['year'].unique()

###
# For each flow, make a layer for each of the 4 from mfa4, each of the 13 from mfa13
layer_template = {'attributes': {'application': ['rw'],
      'applicationConfig': {},
      'dataset': new_rw_id,
      'default': False,
      'description': '',
      'env': 'production',
      'interactionConfig': reduce(lambda obj, col: setup_interaction_config(obj, col, data_tables[wri_id]), data_tables[wri_id].columns, []),
      'iso': [],
      'layerConfig': {'account': 'wri-rw',
       'body': {'layers': [{'options': {'cartocss': cartocss,
           'cartocss_version': '2.3.0',
           'sql': gen_sql(data_col, table_name, date_col, year, filter_col, filter_val)},
          'type': 'mapnik'}],
        'maxzoom': 18,
        'minzoom': 3}},
      'legendConfig': {'items': legend,
       'type': 'choropleth'},
      'name': '',
      'protected': False,
      'provider': 'cartodb'},
     'id': '',
     'type': 'layer'}

    if slider:
        layer_template['attributes']['layerConfig']['timeline'] = True
        layer_template['attributes']['layerConfig']['order'] = int(year)
        layer_template['attributes']['layerConfig']['timelineLabel'] = str(year)

{'attributes': {'application': ['rw'],
  'applicationConfig': {},
  'dataset': new_rw_id,
  'default': True,
  'description': 'The total weight of {flow} of {material} in {year} by country.',
  'env': 'production',
  'interactionConfig': {'output': []},
  'iso': [],
  'language': 'en',
  'layerConfig': {'account': 'wri-rw',
   'body': {'layers': [{'options': {'cartocss': '#com_009_flowmfa {polygon-opacity: 1; line-width: 0.5; line-color: #FFF; line-opacity: 1;} [value<100000]{polygon-fill:#f2f0f7 ;} [value>=100000][value<250000]{polygon-fill:#dadaeb ;} [value>=250000][value<500000]{polygon-fill:#bcbddc ;} [value>=500000][value<1000000]{polygon-fill:#9e9ac8 ;} [value>=1000000][value<3000000]{polygon-fill:#756bb1 ;} [value>=3000000][value<25000000]{polygon-fill:#54278f ;}',
       'cartocss_version': '2.3.0',
       'sql': 'SELECT geo.the_geom_webmercator, geo.cartodb_id, d.isoalpha3, d.flow, d.mfa13, d.time, d.sum_value FROM (SELECT isoalpha3, flow, mfa13, time, SUM(value) as sum_value FROM com_009_flowmfa WHERE flow ILIKE "DMC" AND mfa13 ILIKE "GBF" AND time = 2017 GROUP BY mfa13, isoalpha3, flow, time) d LEFT OUTER JOIN "wri-rw".wri_countries_a geo ON geo.iso_a3 = d.isoalpha3ORDER BY d.isoalpha3'},
      'type': 'mapnik'}],
    'maxzoom': 18,
    'minzoom': 3}},
  'legendConfig': {'items': [{'color': '#f2f0f7', 'name': '<100'},
    {'color': '#dadaeb', 'name': '<250'},
    {'color': '#bcbddc', 'name': '<500'},
    {'color': '#9e9ac8', 'name': '<1000'},
    {'color': '#756bb1', 'name': '<3000'},
    {'color': '#54278f', 'name': '<25000'}],
   'type': 'choropleth'},
  'name': '2017 Domestic Extraction - Non-Metallic Minerals (tonnes, millions)',
  'protected': False,
  'provider': 'cartodb',
  'slug': '2017-Domestic-Extraction-of-Raw-Materials',
  'staticImageConfig': {},
  'updatedAt': '2018-03-13T18:33:31.335Z',
  'userId': '5981e73b0c069f3c93dc5e2a'},
 'id': 'a7dbbf23-254a-49a3-a5dd-b12378fa345b',
 'type': 'layer'}



for layer in old_ds_info.loc['layers'].values[0]:
    layer['attributes']['language'] = 'en'
    layer['attributes']['application'] = ['rw']    
    layer['attributes']['layerConfig']['body']['layers'][0]['options']['cartocss'] = layer['attributes']['layerConfig']['body']['layers'][0]['options']['cartocss'].replace(old_table_name, new_table_name)
    layer['attributes']['layerConfig']['body']['layers'][0]['options']['sql'] = '' + \
    'SELECT geo.the_geom_webmercator, geo.cartodb_id, d.isoalpha3, d.flow, d.mfa4, d.time, d.sum_value FROM ' + \
    '(SELECT isoalpha3, flow, mfa4, time, SUM(value) as sum_value ' + \
    'FROM com_009_flowmfa WHERE flow ILIKE "DMC" AND mfa4 ILIKE "BM" AND time = 2017 ' + \
    'GROUP BY mfa4, isoalpha3, flow, time) d ' + \
    'LEFT OUTER JOIN "wri-rw".wri_countries_a geo ' + \
    'ON geo.iso_a3 = d.isoalpha3' + \
    'ORDER BY d.isoalpha3'
    
    _url = url.format(rw_id = new_rw_id, endpoint = 'layer')
    res = req.request("POST", _url, data=json.dumps(layer['attributes']), headers = headers)
    logging.info(res.text)  

In [ ]:
old_ds_info.loc['layers'].values[0][0]